# Building wind suitablity layers for GRIDCERF

The following code was used to build the wind suitability layers for GRIDCERF. GRIDCERF does not provide the source data directly due to some license restrictions related for direct redistribution of the unaltered source data.  However, the following details the provenance associated with each source dataset and how they were processed.


## 1. Downloading the data

### 1.1 Download GRIDCERF

Download the GRIDCERF package if you have not yet done so from here:  https://doi.org/10.5281/zenodo.6601789.  Please extract GRIDCERF inside the `data` directory of this repository as the paths in this notebook are set to that expectation.


### 1.2 Download the wind potential data


- **Title**:   The Wind Integration National Dataset (WIND) Toolkit  
- **Description from Source**:   Provides wind resource and NREL model data for analysis of the potential for wind energy.
- **Source URL**:  https://gds-files.nrelcloud.org/archive/wind-prospector.zip
- **Date Accessed**:  10/14/22
- **Citation**
> Draxl, C., Clifton, A., Hodge, B.-M. & McCaa, J. The Wind Integration National Dataset (WIND) Toolkit. Applied Energy 151, 355–366 (2015).


## 2. Setup environment

### 2.1 Install GDAL

This application requires GDAL to be installed.  We will call GDAL directly from your command prompt or terminal, so please ensure that you can do so before running the following cells.  More information on how to install GDAL can be found here:  https://gdal.org/download.html


### 2.2 Import necessary Python packages

In [7]:
import os
import glob

import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd


## 3. Configuration

In [27]:
# get the parent directory path to where this notebook is currently stored
root_dir = os.path.dirname(os.getcwd())

# data directory in repository
data_dir = os.path.join(root_dir, "data")

# GRIDCERF data directory from downloaded archive
gridcerf_dir = os.path.join(data_dir, "gridcerf")

# GRIDCERF reference data directory
reference_dir = os.path.join(gridcerf_dir, "reference")

# GRIDCERF common data directory
common_dir = os.path.join(gridcerf_dir, "common")

# GRIDCERF technology_specific data directory
technology_specific_dir = os.path.join(gridcerf_dir, "technology_specific")

# GRIDCERF compiled final suitability data directory
compiled_dir = os.path.join(gridcerf_dir, "compiled")

# template land mask raster
template_raster = os.path.join(reference_dir, "gridcerf_landmask.tif")

# source wind data directory
wind_source_dir = os.path.join(gridcerf_dir, "source", "technology_specific", "nrel_wind")

# Field in wind potential shapefiles that represent the area per polygon that is 
#  available for supporting wind turbines when considering a gross capacity factor
#  of 35%. 
target_wind_area_field = "AREA WITH "

# target fields to include when conducting rasterization
target_wind_fields = [target_wind_area_field, "geometry"]

# generate a list of all common exclusion files
common_raster_list = glob.glob(os.path.join(common_dir, "*.tif"))

# list of technology specific files to use as exclusion areas
technology_specific_list = [os.path.join(technology_specific_dir, "gridcerf_srtm_slope_20pct_or_less.tif")]

# list of suitability files to apply to the wind raster
suitability_file_list = common_raster_list + technology_specific_list


## 4. Generate wind suitability rasters

### 4.1 Functions to build suitability

In [49]:
def get_crs(template_raster: str):
    
    with rasterio.open(template_raster) as src:
        return src.crs


def filter_wind_potential_shapefile(hub_height: int, 
                                    wind_source_dir: str,
                                    target_wind_area_field: str,
                                    target_wind_fields: list,
                                    target_crs: str):
    
    # construct the target wind potential file name
    root_name = f"{hub_height}_Meter_Hub_Height_Current_Technology_"
    target_wind_file = os.path.join(wind_source_dir, root_name, f"{root_name}.shp")

    # read in wind potential shapefile as a GeoPandas data frame
    gdf = gpd.read_file(target_wind_file)[target_wind_fields]
    
    # only keep polygons having an area capable of supporting 35% GCF of >= 1km
    gdf = gdf.loc[gdf[target_wind_area_field] >= 1].copy()
    
    # set target value for rasterization
    gdf["value"] = 0
    
    # reproject to the default GRIDCERF coordinate reference system
    gdf.to_crs(crs=target_crs, inplace=True)
    
    # save filtered shapefile
    basename = f"gridcerf_nrel_wind_development_potential_hubheight{hub_height}m_cf35"
    output_shpfile = os.path.join(wind_source_dir, f"{basename}.shp")
    gdf[["value", "geometry"]].to_file(output_shpfile)
    
    return output_shpfile, basename


def rasterize_potential_noexclusions(input_shapefile: str, 
                                     file_basename: str,
                                     output_dir: str):

    # construct output raster name and path
    output_raster = os.path.join(output_dir, f"{file_basename}_no-exclusions.tif")

    # construct command to rasterize the shapefile using GDAL
    gdal_rasterize_cmd = f"gdal_rasterize -l {file_basename} -a value -tr 1000.0 1000.0 -init 1.0 -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -ot Int16 -of GTiff {input_shapefile} {output_raster}"

    # execute the GDAL command
    print(gdal_rasterize_cmd, '\n')
    os.system(gdal_rasterize_cmd)
    
    return output_raster


def construct_wind_suitability(hub_height: int,
                               input_wind_raster: str, 
                               output_suitability_dir: str,
                               suitability_file_list: list):
    
    # open input wind potential raster
    with rasterio.open(input_wind_raster) as wind_potential_raster:

        # update metadata datatype to int16
        metadata = wind_potential_raster.meta.copy()
        metadata.update({'dtype': rasterio.int16})

        # get array of potential suitability
        wind_suitability_array = wind_potential_raster.read(1)

    # apply suitability rasters
    for idx, i in enumerate(suitability_file_list):
        with rasterio.open(i) as src:
            wind_suitability_array += src.read(1)

    # reclassify to 0 (suitable), 1 (unsuitable)
    wind_suitability_array = np.where(wind_suitability_array == 0, 0, 1)

    # write aggregate tech-specific suitability layer to file
    final_raster = os.path.join(output_suitability_dir, f'gridcerf_wind_onshore_{hub_height}.tif')
    with rasterio.open(final_raster, 'w', **metadata) as dest:
        dest.write(wind_suitability_array.astype(rasterio.int16), 1)
        
    print(f"Generated wind suitability raster:  {final_raster}")
        
    return final_raster

    

### 4.2 Generate wind suitability rasters for all desired hub heights

In [52]:
# list of hub heights to process
hub_height_list = [80, 110, 140]

# get coordinate reference system from template GRIDCERF raster
target_crs = get_crs(template_raster)

# process each hub height
for hub_height in hub_height_list:
    
    print(f"\nProcessing hub height:  {hub_height}...")

    # create a filtered shapefile containing all wind potential polygons having >= 1km of area able to support a GCF of 35%
    filtered_shapefile, basename = filter_wind_potential_shapefile(hub_height=hub_height, 
                                                                   wind_source_dir=wind_source_dir,
                                                                   target_wind_area_field=target_wind_area_field,
                                                                   target_wind_fields=target_wind_fields,
                                                                   target_crs=target_crs)

    # rasterize the shapefile into GRIDCERF's format
    wind_potential_raster = rasterize_potential_noexclusions(input_shapefile=filtered_shapefile,
                                                             file_basename=basename,
                                                             output_dir=technology_specific_dir)
    
    # generate wind suitability output including suitability exclusions
    wind_suitability_raster_file = construct_wind_suitability(hub_height=hub_height,
                                                              input_wind_raster=wind_potential_raster, 
                                                              output_suitability_dir=compiled_dir,
                                                              suitability_file_list=suitability_file_list)


Processing hub height:  80...
gdal_rasterize -l gridcerf_nrel_wind_development_potential_hubheight80m_cf35 -a value -tr 1000.0 1000.0 -init 1.0 -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -ot Int16 -of GTiff /Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf/source/wind/gridcerf_nrel_wind_development_potential_hubheight80m_cf35.shp /Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf/source/wind/gridcerf_nrel_wind_development_potential_hubheight80m_cf35_no-exclusions.tif 

0...10...20...30...40...50...60...70...80...90...100 - done.
Generated wind suitability raster:  /Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf/compiled/gridcerf_wind_onshore_80.tif
Processing hub height:  110...
gdal_rasterize -l gridcerf_nrel_wind_development_potential_hubheight110m_cf35 -a value -tr 1000.0 1000.0 -init 1.0 -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -ot Int16 -of GTiff /Users/d3y010/repos/metarepos/vernon-etal_2022